Вам предлагается предсказать цену квадратного метра недвижимости, используя датасет оценок стоимости квартир
Также разрешается использовать только алгоритм LinearRegression из python пакета sklearn. Использовать другие модели при решении нельзя. Разрешается и рекомендуется придумывать дополнительные признаки для датасета, использовать признаки из сторонних источников и находить наилучшие гиперпараметры алгоритма

In [1]:
import copy
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import sqrt 

from tqdm import tqdm

from sklearn.datasets import *
from sklearn.linear_model import LinearRegression

In [3]:
df_train_init = pd.read_csv('data/andan-2022-autumn-lab-4/train.csv')
df_train_init.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type
0,Ставрополь,NaN,COL_65683,45.011356,41.919273,7,71,144,225,0,...,969.0,8.439655,9.250000,1990.232759,1993.500000,Ставропольский край,31.0,S1842,2020-02-23,10
1,Омск,NaN,COL_159011,54.984833,73.387344,4,40,109,206,0,...,305.0,4.756410,4.333333,1960.346154,1956.291667,Омская область,150.0,S14290,2020-05-24,10
2,Белгород,NaN,COL_205850,50.576805,36.563267,0,9,22,44,0,...,533.0,6.435897,6.400000,1980.452991,1989.633333,Белгородская область,73.0,S25733,2020-06-28,10
3,Большой Исток,NaN,COL_229491,56.714413,60.792454,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,Свердловская область,438.0,S6985,2020-07-19,110
4,Новосибирск,1.0,COL_77283,55.055200,82.895810,0,25,50,101,0,...,1188.0,6.830065,6.672414,1974.045752,1974.637931,Новосибирская область,62.7,S4778,2020-03-01,10


In [4]:
df_train_init.describe()

,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,realty_type
count,76209.000000,206474.000000,206474.000000,206474.000000,206474.000000,206474.000000,206474.000000,206474.000000,206474.000000,206474.000000,...,206474.000000,206474.000000,195541.000000,186141.000000,193961.000000,183936.000000,194311.000000,184340.000000,206474.000000,206474.000000
mean,1.856802,54.343090,47.505203,2.686672,40.283300,80.950328,132.200800,0.038155,0.891488,2.058400,...,99.519920,29.842290,2031.848114,641.936210,7.071113,7.383622,1967.588267,1968.017046,509.704513,54.850926
std,5.416645,4.255719,16.693365,4.180174,52.692116,103.803026,169.929844,0.397521,6.963436,14.904702,...,85.043363,27.591946,1355.356057,445.564108,3.559878,4.259349,46.431030,55.031146,1721.436830,47.864524
min,-3.000000,42.651897,19.892178,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,5.100000,10.000000
25%,1.000000,53.225391,37.583230,0.000000,7.000000,16.000000,28.000000,0.000000,0.000000,0.000000,...,38.000000,9.000000,917.000000,285.000000,4.592233,4.625000,1960.089005,1959.980000,65.000000,10.000000
50%,1.000000,55.677577,39.676300,1.000000,22.000000,46.000000,77.000000,0.000000,0.000000,0.000000,...,85.000000,25.000000,1946.000000,599.000000,6.386792,6.418393,1970.936170,1971.709677,128.000000,10.000000
75%,1.000000,56.296827,55.930234,4.000000,51.000000,100.000000,164.000000,0.000000,0.000000,1.000000,...,139.000000,43.000000,2970.000000,934.000000,8.732143,9.133333,1983.816986,1987.000000,336.975000,110.000000
max,123.000000,69.492140,151.777000,46.000000,454.000000,850.000000,1392.000000,27.000000,586.000000,949.000000,...,733.000000,289.000000,18392.000000,6105.000000,53.717949,221.666667,2019.000000,2020.000000,40000.000000,110.000000


In [5]:
df_train_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206474 entries, 0 to 206473
Data columns (total 76 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   city                                 206474 non-null  object 
 1   floor                                76209 non-null   float64
 2   id                                   206474 non-null  object 
 3   lat                                  206474 non-null  float64
 4   lng                                  206474 non-null  float64
 5   osm_amenity_points_in_0.001          206474 non-null  int64  
 6   osm_amenity_points_in_0.005          206474 non-null  int64  
 7   osm_amenity_points_in_0.0075         206474 non-null  int64  
 8   osm_amenity_points_in_0.01           206474 non-null  int64  
 9   osm_building_points_in_0.001         206474 non-null  int64  
 10  osm_building_points_in_0.005         206474 non-null  int64  
 11  osm_building_

In [6]:
df_test_init = pd.read_csv('data/andan-2022-autumn-lab-4/test_x.csv')
df_test_init.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type
0,Москва,1.0,COL_164098,55.700000,37.752365,5,21,51,89,0,...,1048.0,8.510989,8.791667,1970.950549,1971.250000,Москва,4863.0,S5484,2020-05-24,100
1,Октябрьский,NaN,COL_70629,54.489840,53.490696,0,1,7,21,0,...,136.0,4.937500,3.173913,1973.046875,1965.565217,Башкортостан,26.0,S12273,2020-02-23,100
2,Пермь,1.0,COL_132045,57.969273,56.152996,1,21,37,50,0,...,395.0,7.118644,11.071429,1973.639344,1980.400000,Пермский край,242.0,S34019,2020-04-26,100
3,Балашиха,1.0,COL_16510,55.785270,37.853317,0,5,39,49,0,...,NaN,NaN,NaN,NaN,NaN,Московская область,91.9,S21668,2020-01-19,10
4,"Одинцовский городской округ, с. Аксиньино",3.0,COL_141915,55.738186,36.981342,2,8,8,9,0,...,NaN,NaN,NaN,NaN,NaN,Московская область,1500.0,S2158,2020-05-03,110


In [7]:
df_test_init.describe()

,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,realty_type
count,25317.000000,68825.000000,68825.000000,68825.000000,68825.000000,68825.000000,68825.000000,68825.000000,68825.000000,68825.000000,...,68825.000000,68825.000000,65217.000000,61992.000000,64693.000000,61273.000000,64810.000000,61398.000000,68825.000000,68825.000000
mean,1.934550,54.353278,47.544232,2.714188,40.295765,80.816578,132.182274,0.036586,0.896273,2.082339,...,99.640828,29.851478,2031.103792,641.632388,7.051972,7.365975,1967.712333,1968.272342,508.275451,55.155685
std,5.694383,4.251576,16.737443,4.190849,52.444951,103.064925,168.910192,0.382243,6.730982,14.931919,...,85.025802,27.519762,1355.565429,443.883995,3.536658,4.212348,44.622874,52.297162,1685.407602,47.867757
min,-3.000000,42.711630,19.903135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,5.100000,10.000000
25%,1.000000,53.231277,37.585430,0.000000,7.000000,16.000000,28.000000,0.000000,0.000000,0.000000,...,39.000000,9.000000,915.000000,285.000000,4.590164,4.615385,1960.157529,1959.914286,65.500000,10.000000
50%,1.000000,55.681430,39.692867,1.000000,22.000000,46.000000,77.000000,0.000000,0.000000,0.000000,...,85.000000,24.000000,1946.000000,600.000000,6.369697,6.411765,1970.928571,1971.730249,128.100000,10.000000
75%,1.000000,56.302147,55.893883,4.000000,51.000000,101.000000,165.000000,0.000000,0.000000,1.000000,...,139.000000,43.000000,2977.000000,936.000000,8.717949,9.133333,1983.742629,1987.000000,334.000000,110.000000
max,94.000000,69.500740,136.286820,46.000000,468.000000,850.000000,1390.000000,30.000000,295.000000,730.000000,...,718.000000,284.000000,8034.000000,5933.000000,44.437500,221.666667,2018.600000,2020.000000,40000.000000,110.000000


In [8]:
df_test_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68825 entries, 0 to 68824
Data columns (total 75 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   city                                 68825 non-null  object 
 1   floor                                25317 non-null  float64
 2   id                                   68825 non-null  object 
 3   lat                                  68825 non-null  float64
 4   lng                                  68825 non-null  float64
 5   osm_amenity_points_in_0.001          68825 non-null  int64  
 6   osm_amenity_points_in_0.005          68825 non-null  int64  
 7   osm_amenity_points_in_0.0075         68825 non-null  int64  
 8   osm_amenity_points_in_0.01           68825 non-null  int64  
 9   osm_building_points_in_0.001         68825 non-null  int64  
 10  osm_building_points_in_0.005         68825 non-null  int64  
 11  osm_building_points_in_0.007

In [9]:
df_train_init = df_train_init.drop(['city', 'floor', 'id', 'lat', 'lng', 'osm_city_nearest_name', 'osm_city_nearest_population', 'region', 'street', 'date', 'realty_type'], axis=1)

df_train_init['correlation'] = (df_train_init['osm_catering_points_in_0.005'] + df_train_init['osm_amenity_points_in_0.005'] +  df_train_init['osm_amenity_points_in_0.01'] + df_train_init['osm_amenity_points_in_0.0075'] + df_train_init['osm_catering_points_in_0.0075']) / 5

df_train_init = df_train_init.replace(np.nan, 0)
df_train_init.head()

,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,...,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,correlation
0,7,71,144,225,0,0,0,0,3,23,...,116,32,3090.0,969.0,8.439655,9.250000,1990.232759,1993.500000,31.0,98.8
1,4,40,109,206,0,1,1,1,1,9,...,78,24,1249.0,305.0,4.756410,4.333333,1960.346154,1956.291667,150.0,76.8
2,0,9,22,44,0,1,1,4,0,0,...,120,31,2034.0,533.0,6.435897,6.400000,1980.452991,1989.633333,73.0,15.8
3,0,0,0,1,0,0,0,0,0,0,...,1,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,438.0,0.2
4,0,25,50,101,0,0,0,1,0,5,...,157,58,3777.0,1188.0,6.830065,6.672414,1974.045752,1974.637931,62.7,37.4


In [10]:
df_test_init = df_test_init.drop(['city', 'floor', 'id', 'lat', 'lng', 'osm_city_nearest_name', 'osm_city_nearest_population', 'region', 'street', 'date', 'realty_type'], axis=1)

df_test_init['correlation'] = (df_test_init['osm_catering_points_in_0.005'] + df_test_init['osm_amenity_points_in_0.005'] +  df_test_init['osm_amenity_points_in_0.01'] + df_test_init['osm_amenity_points_in_0.0075'] + df_test_init['osm_catering_points_in_0.0075']) / 5

df_test_init = df_test_init.replace(np.nan, 0)
df_test_init.head()

,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,...,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,correlation
0,5,21,51,89,0,0,0,2,2,2,...,203,56,4247.0,1048.0,8.510989,8.791667,1970.950549,1971.250000,4863.0,33.2
1,0,1,7,21,0,0,1,1,0,0,...,70,24,829.0,136.0,4.937500,3.173913,1973.046875,1965.565217,26.0,6.2
2,1,21,37,50,0,0,0,0,0,3,...,61,15,1236.0,395.0,7.118644,11.071429,1973.639344,1980.400000,242.0,23.2
3,0,5,39,49,0,0,0,0,0,1,...,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,91.9,20.4
4,2,8,8,9,0,0,0,0,1,2,...,0,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1500.0,5.8


In [11]:
correlation = df_train_init.corr().abs().get('per_square_meter_price').sort_values(ascending=False)
correlation = correlation.drop(['per_square_meter_price'])
correlation = correlation[correlation >= 0.1]
correlation

osm_catering_points_in_0.005           0.496335
correlation                            0.492704
osm_amenity_points_in_0.005            0.489408
osm_amenity_points_in_0.01             0.483147
osm_amenity_points_in_0.0075           0.482451
osm_catering_points_in_0.0075          0.481220
osm_catering_points_in_0.01            0.472006
osm_leisure_points_in_0.01             0.464199
osm_healthcare_points_in_0.01          0.463089
osm_shops_points_in_0.01               0.448706
osm_healthcare_points_in_0.0075        0.442801
osm_shops_points_in_0.0075             0.441139
osm_shops_points_in_0.005              0.440054
osm_amenity_points_in_0.001            0.439348
osm_crossing_points_in_0.01            0.435145
osm_transport_stop_points_in_0.01      0.433152
osm_finance_points_in_0.01             0.432395
osm_catering_points_in_0.001           0.428957
osm_offices_points_in_0.01             0.427026
osm_leisure_points_in_0.0075           0.422961
osm_finance_points_in_0.0075           0

In [12]:
def unpow(num):
    count = 0
    while num > 1:
        num = num / 10
        count = count + 1
    return count

for key in correlation.keys():
    maxElement = df_train_init[key].max()
    count =  pow(0.1 ,unpow(maxElement))
    df_train_init[key] = df_train_init[key] * count

In [13]:
train_y = df_train_init['per_square_meter_price'].to_numpy()
df_train_init = df_train_init.drop(['per_square_meter_price'], axis=1)
train_x = df_train_init.to_numpy()

In [14]:
LR = LinearRegression()
LR.fit(train_x, train_y)

LinearRegression()

In [15]:
size = df_test_init.shape[0]
indices = []
predicted = []

In [16]:
for i in tqdm(range(0, size)):
    indices = np.append(indices, i)
    test = df_test_init.iloc[i].to_numpy().reshape(1, -1)
    res = LR.predict(test)
    predicted = np.append(predicted, res)

100%|██████████████████████████████████████████████████████████████████████████| 68825/68825 [01:01<00:00, 1111.04it/s]


In [17]:
df_res = pd.DataFrame({'id': indices.astype(int), 'per_square_meter_price': predicted})
df_res.head()

,id,per_square_meter_price
0,0,5.242903e+09
1,1,9.527566e+08
2,2,1.371754e+09
3,3,4.758359e+07
4,4,2.323248e+07


In [18]:
df_res.to_csv('sample_submission.csv', index=False)

,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,...,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,total_square,correlation
0,0.07,0.071,0.144,0.0225,0,0,0,0,0.03,0.023,...,0.116,32,0.03090,0.0969,0.084397,0.009250,1990.232759,1993.500000,31.00,0.0988
1,0.04,0.040,0.109,0.0206,0,1,1,1,0.01,0.009,...,0.078,24,0.01249,0.0305,0.047564,0.004333,1960.346154,1956.291667,150.00,0.0768
2,0.00,0.009,0.022,0.0044,0,1,1,4,0.00,0.000,...,0.120,31,0.02034,0.0533,0.064359,0.006400,1980.452991,1989.633333,73.00,0.0158
3,0.00,0.000,0.000,0.0001,0,0,0,0,0.00,0.000,...,0.001,0,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,438.00,0.0002
4,0.00,0.025,0.050,0.0101,0,0,0,1,0.00,0.005,...,0.157,58,0.03777,0.1188,0.068301,0.006672,1974.045752,1974.637931,62.70,0.0374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206469,0.02,0.032,0.056,0.0089,0,0,0,0,0.00,0.008,...,0.133,56,0.02371,0.1031,0.054667,0.005560,1966.519380,1967.381818,41.00,0.0396
206470,0.01,0.006,0.008,0.0028,0,0,0,0,0.00,0.000,...,0.109,39,0.02650,0.0939,0.061204,0.006079,1965.807339,1964.461538,98.30,0.0084
206471,0.13,0.097,0.157,0.0227,0,0,0,0,0.03,0.015,...,0.078,29,0.02285,0.0649,0.088947,0.009214,1997.815789,1995.928571,921.50,0.1030
206472,0.03,0.095,0.274,0.0429,0,2,4,4,0.00,0.018,...,0.118,23,0.03806,0.1224,0.063868,0.008056,1961.481132,1962.333333,40.01,0.1742
